In [130]:
import numpy as np
from scipy.io import loadmat
import open3d
import scipy
from scipy.spatial import distance
from open3d import geometry

def affine_transform(pc, R, t):
    return np.array([R.dot(a)+t for a in pc])
    
def background_removal(a_1):
    #geometry.statistical_outlier_removal(input, nb_neighbors, std_ratio)
    #a_1_point_cloud = open3d.PointCloud()
    #a_1_point_cloud.points = open3d.Vector3dVector(a_1)
    #valid_int = geometry.statistical_outlier_removal(a_1_point_cloud, 200, 0.001)
    valid_int = (a_1[:,2])<1
    a_1 = a_1[valid_int]
    
    return a_1

def rigid_motion(p,q):
    """
    Least-Squares Rigid Motion Using Singular Value Decomposition. 
    (https://igl.ethz.ch/projects/ARAP/svd_rot.pdf) 
    
    (note: so far only for the easy case, where all weights are = 1)
    
    p,q: shape [num_points, 3]
    
    """
    n,d = p.shape
    
    # compute centroids
    p_cen = sum(p)/len(p)
    q_cen = sum(q)/len(q)
    
    # compute centered vectors
    X = np.array([i-p_cen for i in p])
    Y = np.array([i-q_cen for i in q])
    
    # compute covariance matrix 
    W = np.eye(n)
    S =  X.T.dot(W).dot(Y)
    
    # compute sigular value decomposition
    U, _, V = np.linalg.svd(S)
    
    # compute optimal R and t
    M = np.eye(d)
    M[-1,-1] = np.linalg.det(V.T.dot(U.T))
    R = V.T.dot(M).dot(U.T)
    
    t = q_cen - R.dot(p_cen)
    
    return R, t

def rms_error(p, q):
    n = p.shape[0]
    dist = [distance.euclidean(p[i,:], q[i,:]) for i in range(n)]
    return np.sqrt(np.sum(np.power(dist, 2))/n)

def show_fitting_result(pcds_list):
    
    point_clouds_object_list = []
    pc = open3d.PointCloud()
    for i, pcd in enumerate(pcds_list):
        point_clouds_object_list.append(open3d.PointCloud())
        point_clouds_object_list[i].points = open3d.Vector3dVector(pcd)
    
    open3d.draw_geometries(point_clouds_object_list)
"""
def informative_subsampling(file):
    img = cv2.imread(file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    kaze = cv2.KAZE_create()
    kps = kaze.detect(img)
    
    print(len(kps))
    
    # Do something with normals of the images to get either the coordinates or points in pointcloud
    
    return

"""    
def icp(a_1, a_2, convergence_treshold=0.005, point_selection="all", sample_size=500, generate_3d = True, verbose = True, is_test = False , accuracy_check = False, stability_constant = 1,source_file=None , target_file=None):
    """
    a_1: positions of points in point cloud 1. shape : [num_points1, 3]
    a_2: positions of points in point cloud 2. shape : [num_points2, 3]
    
    """
    
    if is_test:
        generate_3d = False
        verbose = False
    n,d = a_1.shape
    
    
    
    # Filter the point clouds based on the depth,
    # only keep the indices where the z of the point cloud is less than 1
    a_1, a_2 = background_removal(a_1), background_removal(a_2)
    
    a_2_c = a_2.copy()
    # Point selection
    # Uniform subsampling
    if point_selection == "uniform":
        a_1 = a_1[np.random.randint(low=0, high=a_1.shape[0], size=sample_size)]
        a_2 = a_2[np.random.randint(low=0, high=a_2.shape[0], size=sample_size)]
        
    if point_selection == "informative":
        a_1 = informative_subsampling(source_file)
        a_2 = informative_subsampling(target_file)
    
    if stability_constant == 1 :
        R_overall = np.eye(d)
        t_overall = np.zeros(d)
    else:
        R_overall = np.random.normal(0,1, size = (d,d))*stability_constant
        t_overall = np.random.normal(0,1, size = d)*stability_constant
    
    # Base loop on difference in rsm error
    rms_error_old = 10000
    rms_error_new = rms_error_old-1
    
    while rms_error_old-rms_error_new > convergence_treshold:
        
        if point_selection == "random":
            a_1 = a_1[np.random.choice(a_1.shape[0], sample_size, replace=False), :]
            a_2 = a_2[np.random.choice(a_2.shape[0], sample_size, replace=False), :]
            
        # (Step 1) Find closest points for each point in a_1 from a_2
        tree = scipy.spatial.KDTree(a_2)
        closest_dists, closest_idx = tree.query(a_1)
        # Found this on stackoverflow: https://bit.ly/2P8IYiw
        # Not sure if we can use this, but it is definetely much (!!) faster 
        # than manually comparing all the vectors.
        # Usage also proposed on Wikipedia: https://bit.ly/2urg9nU
        # For how-to-use see: https://bit.ly/2UbKNfn
        closest_a_2 = a_2[closest_idx]
    
        # (Step 2) Refine R and t using Singular Value Decomposition
        R, t = rigid_motion(a_1,closest_a_2)
       
        # update a_1
        a_1 =affine_transform(a_1, R, t)
        
        # update rms error
        rms_error_old = rms_error_new
        rms_error_new = rms_error(a_1, closest_a_2)
        
        if verbose:
            print(rms_error_new)
        
        # update overall R and t
        R_overall = R.dot(R_overall)
        t_overall = R.dot(t_overall) + t
    if generate_3d:
        show_fitting_result([a_1, a_2_c])
    if accuracy_check:
        return rms_error_new
    return R_overall, t_overall
    

In [57]:
# print("Load a ply point cloud, print it, and render it")
# pcd1  = open3d.read_point_cloud("Data/data/0000000000.pcd")
# pcd2  = open3d.read_point_cloud("Data/data/0000000001.pcd")
# print(np.asarray(pcd.points))
# open3d.draw_geometries([pcd])
#a_1 = loadmat("Data/source.mat")["source"].T
#a_2 = loadmat("Data/target.mat")["target"].T

a_1 = open3d.read_point_cloud("Data/data/0000000000.pcd")
a_2 = open3d.read_point_cloud("Data/data/0000000002.pcd")

a_1 = np.asarray(a_1.points)
a_2 = np.asarray(a_2.points)

#source_file = "Data/data/0000000000.jpg"
#target_file = "Data/data/0000000001.jpg"

In [128]:
t = np.array([1,2,3])
p = t>2
t[p]

array([3])

In [131]:
icp(a_1, a_2, point_selection = 'random')

0.014073748653883857
0.013849565669256235


(array([[ 0.99929638, -0.01050765, -0.03600473],
        [ 0.01035983,  0.99993713, -0.00428955],
        [ 0.03604754,  0.00391353,  0.99934241]]),
 array([ 0.03296695,  0.00338745, -0.0013784 ]))

In [9]:
sampling_methods = ['random', 'uniform', 'default']#, "informative"]

In [14]:
def speed_check(a_1, a_2, sampling_methods):
    for method in sampling_methods:
        print("Time that it takes for {} method is:".format(method))
        %time  icp(a_1, a_2, point_selection = method, is_test = True)
speed_check(a_1, a_2, sampling_methods)

Time that it takes for random method is:
CPU times: user 2.24 s, sys: 32.5 ms, total: 2.27 s
Wall time: 1.6 s
Time that it takes for uniform method is:
CPU times: user 2.33 s, sys: 32 ms, total: 2.36 s
Wall time: 1.72 s
Time that it takes for default method is:
CPU times: user 30.2 s, sys: 2.04 s, total: 32.3 s
Wall time: 31.1 s


In [15]:
def accuracy_check(a_1, a_2, sampling_methods):
    for method in sampling_methods:
        print("RMSE of {} method is {:.4}".format(method, icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True)))
accuracy_check(a_1, a_2, sampling_methods)
    

RMSE of random method is 0.131
RMSE of uniform method is 0.1368
RMSE of default method is 0.05849


In [16]:
def  noise_check(a_1, a_2, sampling_methods):
    noise_1, noise_2 = np.random.normal(0,1,(a_2.shape)) ,np.random.normal(0,1,(a_2.shape))
    a_1_noisy, a_2_noisy = a_1 + noise_1 , a_2 + noise_2
    for method in sampling_methods:
        rmse_normal =icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True) 
        rmse_noisy = icp(a_1_noisy, a_2_noisy, point_selection = method, is_test = True, accuracy_check = True)
        print("RMSE of {} method is {:.4}, whereas if we add noise, it becomes {:.4} , the difference is {:.4}".format(method, rmse_normal, rmse_noisy, rmse_normal - rmse_noisy))
        R_normal , t_normal = icp(a_1, a_2, point_selection = method, is_test = True) 
        R_noisy , t_noisy = icp(a_1_noisy, a_2_noisy, point_selection = method, is_test = True) 
        R_distance, t_distance = np.linalg.norm(R_normal - R_noisy), np.linalg.norm(t_normal - t_noisy)
        print("The distance between normal and noisy R matrices is {:.4} and between normal and noisy t vectors is {:.4}".format(R_distance, t_distance))
        

noise_check(a_1, a_2, sampling_methods)
    

RMSE of random method is 0.1319, whereas if we add noise, it becomes 0.4216 , the difference is -0.2897
The distance between normal and noisy R matrices is 0.4999 and between normal and noisy t vectors is 0.313
RMSE of uniform method is 0.1455, whereas if we add noise, it becomes 0.4292 , the difference is -0.2836
The distance between normal and noisy R matrices is 0.4932 and between normal and noisy t vectors is 0.4255
RMSE of default method is 0.05849, whereas if we add noise, it becomes 0.2044 , the difference is -0.1459
The distance between normal and noisy R matrices is 0.4828 and between normal and noisy t vectors is 0.4542


In [17]:
def  stability_check(a_1, a_2, sampling_methods):
    stability_constant = 10
    for method in sampling_methods:
        rmse_normal =icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True) 
        rmse_non_stable= icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True, stability_constant = stability_constant)
        print("RMSE of {} method is {:.4}, whereas if we have random initialisation, it becomes {:.4} , the difference is {:.4}".format(method, rmse_normal, rmse_non_stable, rmse_normal - rmse_non_stable))
        R_normal , t_normal = icp(a_1, a_2, point_selection = method, is_test = True) 
        R_non_stable , t_non_stable = icp(a_1, a_2, point_selection = method, is_test = True, stability_constant = stability_constant) 
        R_distance, t_distance = np.linalg.norm(R_normal - R_non_stable), np.linalg.norm(t_normal - t_non_stable)
        print("The distance between normal and randomely initialised R matrices is {:.4} and between normal and randomely initialised t vectors is {:.4}".format(R_distance, t_distance))
        

stability_check(a_1, a_2, sampling_methods)
    

RMSE of random method is 0.1279, whereas if we have random initialisation, it becomes 0.142 , the difference is -0.0141
The distance between normal and randomely initialised R matrices is 23.84 and between normal and randomely initialised t vectors is 8.554
RMSE of uniform method is 0.1345, whereas if we have random initialisation, it becomes 0.1289 , the difference is 0.005588
The distance between normal and randomely initialised R matrices is 24.18 and between normal and randomely initialised t vectors is 15.35
RMSE of default method is 0.05849, whereas if we have random initialisation, it becomes 0.05849 , the difference is 0.0
The distance between normal and randomely initialised R matrices is 25.6 and between normal and randomely initialised t vectors is 10.1


Note: Take stability as an example, an experimental setup may be the following: given a source point cloud, augmented target point cloud can be obtained by transforming the source point cloud using a random rigid transform R and t. Stability can be analyzed by observing a behaviour of ICP on source and augmented target point cloud w.r.t changes in magnitude of R and t.
        

- stability is about convergence of the algorithm dependent on initial condition.

- tolerance to noise is about convergence of the algorithm with input data with noise. You can imagine data is captured by a sensor. In the ideal case you will obtain exact point cloud, however sensor is not precise, therefore there will be noise in measurement. Therefore we ask you to evaluate how ICP is robust against those kind of issuses.

In [ ]:

#merging scenes
#icp(a_1, a_2, is_test = True)
#pcd1  = open3d.read_point_cloud("Data/data/0000000000.pcd")
#pcd2  = open3d.read_point_cloud("Data/data/0000000001.pcd")
max_images = 99
def merge_one_after_another(data_directory = "Data/data/", point_selection = 'random', max_images = 99):
    print("Load a ply point cloud, print it, and render it")
    
    pcds= []
    for i in range(max_images+1):
        if i<10:
            pcds.append(open3d.read_point_cloud("{}000000000{}.pcd".format(data_directory, i)).points)
        else:
            pcds.append(open3d.read_point_cloud("{}00000000{}.pcd".format(data_directory, i)).points)
    pcds = list(map(np.asarray, pcds))
    
    R_list,  t_list = [], []
    for i in range(len(pcds)-1):
        R, t = icp(pcds[i], pcds[i+1], is_test = True, point_selection = point_selection)
        R_list.append(R)
        t_list.append(t)
    
    pcds = list(map(background_removal, pcds))
    pcds_transformed = []
    
    for i in range(len(pcds)-1):
        pcds_transformed.append(affine_transform(pcds[i], R_list[i], t_list[i]))

        
    
    show_fitting_result(pcds_transformed)
merge_one_after_another(data_directory = "Data/data/", point_selection = 'random', max_images = 99)

Load a ply point cloud, print it, and render it
